In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('Week3_GA_dataset.csv')
df.head()

,V1,V2,V3,V4,V5,Target
0,2.0,50.0,12500.0,98.0,NEGATIVE,YES
1,0.0,13.0,3250.0,28.0,NEGATIVE,YES
2,?,?,4000.0,35.0,NEGATIVE,YES
3,?,20.0,5000.0,45.0,NEGATIVE,YES
4,1.0,24.0,6000.0,77.0,NEGATIVE,NO


In [ ]:
df.replace('?', np.nan, inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold

num_pipeline = Pipeline([('selector',ColumnTransformer([('select_first_4',
                                                         'passthrough',
                                                         slice(0,4))])),
                         ('imputer', SimpleImputer(strategy="mean")),
                         ('std_scaler', StandardScaler()),
                        ])
cat_pipeline = ColumnTransformer([('ordinal_encoder', OrdinalEncoder(),[4]),
                                  ])

full_pipeline = FeatureUnion(transformer_list=
                             [("num_pipeline", num_pipeline),
                              ("cat_pipeline", cat_pipeline)
                             ])

final_pipeline = Pipeline([
    ("full_pipeline", full_pipeline),
    ("variance_feature_selection", VarianceThreshold(threshold=0.1))
])

In [ ]:
new_data = final_pipeline.fit_transform(df)
new_data.shape

(748, 4)

In [ ]:
num_pipeline = Pipeline([('selector',ColumnTransformer([('select_first_4',
                                                         'passthrough',
                                                         slice(0,4))])),
                         ('imputer', SimpleImputer(strategy="mean")),
                         ('std_scaler', StandardScaler()),
                        ])
cat_pipeline = ColumnTransformer([('ordinal_encoder', OrdinalEncoder(),[4]),
                                  ])

full_pipeline = FeatureUnion(transformer_list=
                             [("num_pipeline", num_pipeline),
                              ("cat_pipeline", cat_pipeline)
                             ])

In [ ]:
new_data = full_pipeline.fit_transform(df)
new_data.shape

(748, 5)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

estimator = LogisticRegression()
selector = RFE(estimator, n_features_to_select = 2, step=1)
selector = selector.fit(new_data, df['Target'])

print(selector.support_)
print(f'Rank of each feature is : {selector.ranking_}')

[ True False  True False False]
Rank of each feature is : [1 3 1 2 4]


In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
sfs = SequentialFeatureSelector(estimator, n_features_to_select=2)
sfs.fit_transform(new_data, df['Target'])
print(sfs.get_support())

[False  True False  True False]


In [ ]:
sfs = SequentialFeatureSelector(estimator, n_features_to_select=2, direction = 'backward')
sfs.fit_transform(new_data, df['Target'])
print(sfs.get_support())

[False False  True  True False]
